In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('./'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also writ

./.amlignore
./.amlignore.amltmp
./0.99.ipynb
./0.99.ipynb.amltmp
./0.99_orig.ipynb
./0.99_orig.ipynb.amltmp
./plot.ipynb
./plot.ipynb.amltmp
./predictions.csv
./substances.csv
./test.csv
./train.csv
./.ipynb_aml_checkpoints/0.99-checkpoint2024-11-18-20-34-41Z.ipynb
./.ipynb_aml_checkpoints/0.99-checkpoint2024-11-18-21-43-44Z.ipynb
./.ipynb_aml_checkpoints/0.99-checkpoint2024-11-18-22-31-27Z.ipynb
./.ipynb_aml_checkpoints/0.99-checkpoint2024-11-18-22-34-50Z.ipynb
./.ipynb_aml_checkpoints/0.99-checkpoint2024-11-18-22-35-3Z.ipynb
./.ipynb_aml_checkpoints/0.99-checkpoint2024-11-19-11-31-26Z.ipynb
./.ipynb_aml_checkpoints/0.99-checkpoint2024-11-19-13-2-6Z.ipynb
./.ipynb_aml_checkpoints/0.99-checkpoint2024-11-19-14-16-15Z.ipynb
./.ipynb_aml_checkpoints/0.99-checkpoint2024-11-19-15-18-37Z.ipynb
./.ipynb_aml_checkpoints/0.99-checkpoint2024-11-19-15-54-1Z.ipynb
./.ipynb_aml_checkpoints/0.99-checkpoint2024-11-19-16-29-19Z.ipynb
./.ipynb_aml_checkpoints/0.99-checkpoint2024-11-19-17-15-53Z.ipynb


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from sklearn.model_selection import GridSearchCV, cross_val_score, RandomizedSearchCV
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.decomposition import PCA
from torch.utils.data import TensorDataset, DataLoader
from scipy.stats import uniform, randint, loguniform
from sklearn import preprocessing

import warnings
warnings.filterwarnings("ignore")

In [3]:
data_train = pd.read_csv("./train.csv")
data_test = pd.read_csv("./test.csv")

X_train = data_train.iloc[:, 6:].values
X_test = data_test.iloc[:, 5:].values

In [4]:
data_train

,sample_name,device_serial,substance_form_display,measure_type_display,prod_substance,PURITY,908.1,914.3,920.5,926.7,...,1620.5,1626.6,1632.8,1639,1645.2,1651.4,1657.6,1663.8,1670,1676.2
0,11140009,M1-1000112,Homogenized Powder,Direct contact,Heroin,58.500000,0.044734,0.042720,0.041361,0.040055,...,0.003972,0.007906,0.012490,0.018123,0.025070,0.033235,0.042502,0.052237,0.061383,0.068823
1,22.0401-P002.02,M1-1000109,Non homogenized powder,Direct contact,Heroin,18.600000,0.063695,0.056980,0.050080,0.043339,...,0.093002,0.099668,0.108805,0.117120,0.121947,0.125137,0.128688,0.133501,0.138187,0.140248
2,22.0117-P001.02,M1-1000100,Non homogenized powder,Direct contact,Heroin,19.881719,0.050358,0.044910,0.039148,0.033730,...,0.083369,0.090485,0.100462,0.109033,0.113411,0.117053,0.121665,0.128366,0.134636,0.136961
3,20.0163-P009,N1-00196,Unspecified,Direct contact,Heroin,52.500000,0.525050,0.520434,0.517187,0.516377,...,0.357223,0.370060,0.386062,0.404460,0.425567,0.450527,0.479066,0.508943,0.539349,0.564486
4,19.0286-P005,N1-00196,Unspecified,Direct contact,Heroin,48.900000,0.479365,0.467401,0.456680,0.446148,...,0.350973,0.366094,0.384536,0.405034,0.426582,0.450564,0.477045,0.504142,0.531764,0.553650
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1295,22.0267-P001.01,M1-1000100,Homogenized Powder,Direct contact,Heroin,10.100000,0.107930,0.103983,0.100068,0.096422,...,0.133917,0.138535,0.145144,0.151008,0.153738,0.155602,0.158525,0.163138,0.167623,0.169048
1296,11060000,M1-1000144,Homogenized Powder,Direct contact,Heroin,44.900000,0.040564,0.041559,0.042435,0.043870,...,-0.007995,-0.004902,-0.001237,0.003390,0.009303,0.016472,0.023843,0.031736,0.039568,0.046056
1297,22.0368-P002,M1-1000130,Homogenized Powder,Direct contact,Heroin,53.400000,0.106459,0.108084,0.109983,0.109541,...,0.052178,0.056051,0.060666,0.066257,0.073435,0.083057,0.094621,0.106288,0.117307,0.126626
1298,22.0342-P002.04,M1-1000109,Homogenized Powder,Direct contact,Heroin,13.600000,0.075628,0.068853,0.062538,0.056079,...,0.094620,0.101527,0.111090,0.119861,0.124868,0.128340,0.132605,0.138737,0.144825,0.148246


In [5]:
pca = PCA()
scaler = StandardScaler() #scale before pca
pca.fit(scaler.fit_transform(X_train))
cumvariance = np.cumsum(pca.explained_variance_ratio_)
cumvariance

array([0.89483039, 0.98432745, 0.99708348, 0.99918118, 0.99964032,
       0.99976243, 0.99985849, 0.99991471, 0.99994053, 0.9999607 ,
       0.99997248, 0.99998097, 0.99998545, 0.99998914, 0.99999149,
       0.99999344, 0.99999478, 0.99999564, 0.99999645, 0.99999708,
       0.99999756, 0.99999796, 0.99999829, 0.99999856, 0.99999879,
       0.99999896, 0.99999906, 0.99999915, 0.99999923, 0.99999929,
       0.99999934, 0.99999939, 0.99999944, 0.99999948, 0.99999951,
       0.99999954, 0.99999957, 0.9999996 , 0.99999962, 0.99999965,
       0.99999967, 0.99999969, 0.99999971, 0.99999972, 0.99999974,
       0.99999976, 0.99999977, 0.99999978, 0.9999998 , 0.99999981,
       0.99999982, 0.99999983, 0.99999984, 0.99999985, 0.99999986,
       0.99999986, 0.99999987, 0.99999988, 0.99999989, 0.99999989,
       0.9999999 , 0.9999999 , 0.99999991, 0.99999991, 0.99999992,
       0.99999992, 0.99999993, 0.99999993, 0.99999994, 0.99999994,
       0.99999994, 0.99999995, 0.99999995, 0.99999995, 0.99999

In [6]:
# keep 99% of variance
#n_comp = np.where(cumvariance>0.99)[0][0]
#n_comp

In [7]:
# pure = pd.read_csv("./substances.csv")
# #pure = pure[pure["substance"].str.contains("heroin")]
# pure = pure.iloc[:,1:].values
# pure.shape

In [8]:
# X_normalized = preprocessing.normalize(X_train, axis=1)
# pure_normalized = preprocessing.normalize(pure, axis=1)

In [9]:
# X_normalized.shape

In [10]:
# X_normalized = X_normalized @ pure_normalized.T
# X_normalized.shape

In [11]:
# pca = PCA()
# pca.fit(X_normalized)
# np.cumsum(pca.explained_variance_ratio_)

In [12]:
y = data_train.iloc[:,5:6].values/100
y.shape

(1300, 1)

In [13]:

seed = 43
"""
# Combine original data with PCA components
standardizer = StandardScaler()
X_st = standardizer.fit_transform(X_train)
"""

# Standardisation des données (avant PCA)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


## Perform PCA 
pca = PCA()
X_train= pca.fit_transform(X_train)
X_test = pca.transform(X_test)

#X_normalized = preprocessing.normalize(X_pca)
#X_normalized = X_train



#X_train, X_valid, y_train, y_valid = train_test_split(X_normalized, y, test_size=0.2, random_state = 42)

# Convert to tensors
#X_train_tensor = torch.tensor(X_train, dtype=torch.float32).clone().detach()
#y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).reshape(-1, 1).clone().detach()
#X_valid_tensor = torch.tensor(X_valid, dtype=torch.float32).clone().detach()
#y_valid_tensor = torch.tensor(y_valid.values, dtype=torch.float32).reshape(-1, 1).clone().detach()

In [14]:
X_train

array([[-4.00028256e+00, -3.42618089e+00,  3.11611074e-01, ...,
        -8.72434355e-05,  5.62935897e-05, -8.33068381e-05],
       [-2.26440251e+00, -4.57058600e-01, -8.12102112e-01, ...,
         6.63679017e-06, -6.25942360e-06,  1.09093150e-04],
       [-3.08935772e+00, -5.98678213e-01, -7.18662885e-01, ...,
        -2.12820618e-04,  3.26064102e-05,  1.47445781e-04],
       ...,
       [ 2.28568412e+00, -3.76359106e+00,  4.25107901e-01, ...,
        -2.33480405e-05, -2.05388898e-04, -6.43270285e-05],
       [-1.81250742e+00, -5.65106786e-01, -5.37556900e-01, ...,
         1.79559475e-04,  2.19990433e-05,  6.85000964e-05],
       [-3.00206406e+00, -5.71250127e-01, -7.41033143e-01, ...,
        -5.64405609e-05, -4.97562466e-05,  4.17144131e-05]])

In [15]:
y_train = y
y_train

array([[0.585     ],
       [0.186     ],
       [0.19881719],
       ...,
       [0.534     ],
       [0.136     ],
       [0.196     ]])

In [16]:
#X_pca = pca.transform(X_test)
#X_test_normalized = preprocessing.normalize(X_pca)
#X_test_normalized = X_test
#X_test_tensor = torch.tensor(X_test_normalised, dtype=torch.float32).clone().detach() 

# X_test_normalized = preprocessing.normalize(X_test.values, axis=1)
# X_test_normalized = X_test_normalized @ pure_normalized.T
# X_test_normalized.shape

In [17]:
def my_loss(output, target):
    loss = torch.maximum(torch.zeros_like(output),(output - target)-.05) + torch.maximum(torch.zeros_like(output),(target - output)-.05)
    return torch.mean(loss)

In [18]:
# Définir le modèle de réseau de neurones simple

class FeedForwardNN(nn.Module):

    def __init__(self, input_size, lin_layer_sizes,

                 outpout_size, lin_layer_dropouts, activation):
        super().__init__()       
        if activation == 0:
            self.activation = nn.ReLU()
        elif activation == 1:
            self.activation = nn.SiLU()
        elif activation == 2:
            self.activation = nn.Tanh()
        elif activation == 3:
            self.activation = nn.LeakyReLU()  

        self.input_size = input_size

        # Linear Layers
        first_lin_layer = nn.Linear(input_size, lin_layer_sizes[0])
        self.lin_layers = nn.ModuleList([first_lin_layer] + [nn.Linear(lin_layer_sizes[i], lin_layer_sizes[i + 1]) for i in range(len(lin_layer_sizes) - 1)])     

        # Output Layer
        self.outpout_layer = nn.Linear(lin_layer_sizes[-1], outpout_size)
        
        # Dropout Layers
        self.dropout_layers = nn.ModuleList([nn.Dropout(rate) for rate,size in zip(lin_layer_dropouts,lin_layer_sizes)])

    def forward(self, x):
        for lin_layer, dropout_layer in zip(self.lin_layers, self.dropout_layers):
            x = lin_layer(x)     
            x = self.activation(x)
            x = dropout_layer(x)
 
        x = self.outpout_layer(x)
        x = nn.Sigmoid()(x)
  
        return x

# Définir la classe NeuralNetRegressor
class NeuralNetRegressor(BaseEstimator, RegressorMixin):

    def __init__(self, input_size, random_state, eta=0.001, max_epochs=100, batch=10, lin_layer_sizes = [50, 50],
            outpout_size = 1, lin_layer_dropouts = [0.4, 0.4], activation = 0, loss=0, n_comp=None):

        self.input_size = input_size
        self.random_state = random_state
        self.eta = eta
        self.max_epochs = max_epochs
        self.batch = batch
        self.lin_layer_sizes = lin_layer_sizes
        self.outpout_size = outpout_size
        self.lin_layer_dropouts = lin_layer_dropouts
        self.activation = activation
        self.model = None

        self.n_comp = n_comp

        self.loss = loss
       
    def fit(self, X, y, do_print=False,X_valid=None, y_valid=None ):
        #print("self.eta",self.eta)
        #print("X",X.shape)
        #print("Y",y.shape)

        if self.n_comp is not None:
            print("n_comp", self.n_comp)
            self.input_size = self.n_comp

        #print("FITTING")
        if self.loss==0:
            self.criterion = nn.MSELoss()
        elif self.loss==1:
            self.criterion = nn.L1Loss()
        else:
            self.criterion = my_loss

        self.model = FeedForwardNN(self.input_size, self.lin_layer_sizes,
                 self.outpout_size, self.lin_layer_dropouts, self.activation)


        #print("n_comp", self.n_comp)
        if self.n_comp is not None:
            X = X[:,:self.n_comp]

        #print("shape", X.shape)
        
        #print("model", self.model)

        optimizer = optim.Adam(self.model.parameters(), lr=self.eta)
        X_tensor = torch.tensor(X).clone().detach().float()
        y_tensor = torch.tensor(y).clone().detach().float()
        dataset = TensorDataset(X_tensor, y_tensor)
        dataloader = DataLoader(dataset, batch_size=self.batch, shuffle=True)
        self.model.train()

        if X_valid is not None:
            X_valid = torch.tensor(X_valid).clone().detach().float()
            y_valid = torch.tensor(y_valid).clone().detach().float()

        # Training loop
        for epoch in range(self.max_epochs):

            epoch_loss = 0.0
            for batch_X, batch_y in dataloader:
                optimizer.zero_grad()  # Reset gradients
                outputs = self.model(batch_X)  # Forward pass
                loss = self.criterion(outputs, batch_y)  # Compute loss
                loss.backward()  # Backward pass
                optimizer.step()  # Update parameters
                epoch_loss += loss.item()
            if do_print and X_valid is not None:
                outputs = self.model(X_valid)  # Forward pass
                valid_loss = self.criterion(outputs, y_valid)  # Compute loss
                print(f"Epoch {epoch+1}/{self.max_epochs}, Loss: {epoch_loss / len(dataloader)}, Valid_loss: {valid_loss}")

            elif do_print:
                print(f"Epoch {epoch+1}/{self.max_epochs}, Loss: {epoch_loss / len(dataloader)}")

        return self  

    def predict(self, X):
        self.model.eval()
        print("n_comp", self.n_comp)
        if self.n_comp is not None:
            X = X[:,:self.n_comp]

        X_tensor = torch.tensor(X, dtype=torch.float32)
        with torch.no_grad():
            outputs = self.model(X_tensor)
        return outputs.numpy()
        
    def parameters(self):
        return self.model.parameters()

In [19]:
class RandomLayers:
    def __init__(self,min_layers, max_layers, min_nodes, max_nodes):
        self.min_nodes = min_nodes
        self.max_nodes = max_nodes
        self.min_layers = min_layers
        self.max_layers = max_layers

    def rvs(self, random_state=None):
        if random_state is not None:
            prev = randint.random_state
            randint.random_state = random_state 

        nodes = randint.rvs(self.min_nodes, self.max_nodes)
        res = [nodes for _ in range(randint.rvs(self.min_layers, self.max_layers))]

        if random_state is not None:
            randint.random_state = prev

        return res

In [20]:
class RandomDropout:
    def __init__(self, max_layers,min_dropout,max_dropout):
        self.min_dropout = min_dropout
        self.max_dropout = max_dropout
        self.max_layers = max_layers

    def rvs(self, random_state=None):
        if random_state is not None:
            prev = uniform.random_state
            uniform.random_state = random_state 
        dropout = uniform.rvs(self.min_dropout, self.max_dropout)
        res = [dropout for _ in range(self.max_layers)]

        if random_state is not None:
            uniform.random_state = prev

        return res

In [21]:

def my_custom_score(ground_truth, predictions):
    return np.mean(np.abs(predictions-ground_truth)*100<=5)


from sklearn.metrics import make_scorer

scorer = make_scorer(my_custom_score, greater_is_better=True)

In [22]:
# Initialiser le modèle

seed = 42
np.random.seed(seed)
torch.manual_seed(seed)
input_size = X_train.shape[1]
net = NeuralNetRegressor(input_size=input_size, random_state=seed)

max_layers = 5

# Définir les paramètres pour GridSearch
params_dist = {

    'eta': loguniform(1e-4, 1e-1),
    'max_epochs': randint(10, 500),
    'batch': randint(8, 64),
    'lin_layer_sizes': RandomLayers(1,max_layers,4,256),  # Taille de 1 à 4 couches, entre 4 et 256 neurones par couche
    'lin_layer_dropouts': RandomDropout(max_layers,0.,.5),
    'activation': randint(0, 4),
    'loss': randint(0, 3),
    'n_comp': randint(1, X_train.shape[1]),
}

# Initialiser RandomizedSearchCV
random_search = RandomizedSearchCV(net, params_dist, refit=True, cv=10, random_state=seed, scoring=scorer, verbose=3, n_iter=120, n_jobs=-2)

# Entraîner le modèle avec GridSearch
random_grid_result = random_search.fit(X_train, y_train)
nouveau_model = random_grid_result.best_estimator_
print("Best score: %f using %s" % (random_grid_result.best_score_, random_grid_result.best_params_))

Fitting 10 folds for each of 120 candidates, totalling 1200 fits


In [ ]:
# from matplotlib import pyplot as plt
# def plot_search_results(grid):
#     """
#     Params: 
#         grid: A trained GridSearchCV object.
#     """
#     ## Results from grid search
#     results = grid.cv_results_
#     means_test = results['mean_test_score']
#     stds_test = results['std_test_score']
#     #means_train = results['mean_train_score']
#     #stds_train = results['std_train_score']

#     ## Getting indexes of values per hyper-parameter
#     masks=[]
#     masks_names= list(grid.best_params_.keys())
#     for p_k, p_v in grid.best_params_.items():
#         masks.append(list(results['param_'+p_k].data==p_v))

#     params=grid.param_grid

#     ## Ploting results
#     fig, ax = plt.subplots(1,len(params),sharex='none', sharey='all',figsize=(20,5))
#     fig.suptitle('Score per parameter')
#     fig.text(0.04, 0.5, 'MEAN SCORE', va='center', rotation='vertical')
#     pram_preformace_in_best = {}
#     for i, p in enumerate(masks_names):
#         m = np.stack(masks[:i] + masks[i+1:])
#         pram_preformace_in_best
#         best_parms_mask = m.all(axis=0)
#         best_index = np.where(best_parms_mask)[0]
#         x = np.array(params[p])
#         y_1 = np.array(means_test[best_index])
#         e_1 = np.array(stds_test[best_index])
#         #y_2 = np.array(means_train[best_index])
#         #e_2 = np.array(stds_train[best_index])
#         ax[i].errorbar(x, y_1, e_1, linestyle='--', marker='o', label='test')
#         #ax[i].errorbar(x, y_2, e_2, linestyle='-', marker='^',label='train' )
#         ax[i].set_xlabel(p.upper())

#     plt.legend()
#     plt.show()

In [ ]:
# plot_search_results(random_search)

In [37]:
random_search.best_params_

{'activation': 3,
 'batch': 27,
 'eta': 0.00022722376351800997,
 'lin_layer_dropouts': [0.05876312338855244,
  0.05876312338855244,
  0.05876312338855244,
  0.05876312338855244,
  0.05876312338855244],
 'lin_layer_sizes': [108, 108, 108],
 'loss': 0,
 'max_epochs': 396,
 'n_comp': 77}

In [34]:
random_search.best_estimator_

NeuralNetRegressor(activation=3, batch=27, eta=0.00022722376351800997,
                   input_size=77,
                   lin_layer_dropouts=[0.05876312338855244, 0.05876312338855244,
                                       0.05876312338855244, 0.05876312338855244,
                                       0.05876312338855244],
                   lin_layer_sizes=[108, 108, 108], max_epochs=396, n_comp=77,
                   random_state=42)

In [35]:
random_search.best_score_

0.9715384615384615

In [ ]:
# nnreg = NeuralNetRegressor(input_size=input_size, random_state=55, **random_grid_result.best_params_)
# nnreg.fit(X_train, y_train,do_print=True, X_valid=X_valid, y_valid=y_valid )

In [ ]:
# nnreg.model.eval()
# y_pred = nnreg.model(torch.tensor(X_valid).float())

# #np.mean(np.square(y_valid-y_pred))

In [ ]:
# y_pred2 = nnreg.predict(X_valid)
# y_pred2 - y_pred.detach().numpy()[:,0]


In [ ]:
# loss = nn.MSELoss()
# loss(torch.tensor(y_valid),torch.tensor(y_pred))

In [36]:
#y_pred = nouveau_model.predict(X_valid)
y_pred_train = nouveau_model.predict(X_train)
predictions = nouveau_model.predict(X_test)*100

# Vérifiez les sorties du modèle
print("y_pred contains NaN:", np.isnan(predictions).any())



# Calculer le t_score
train_score = np.mean(np.abs(y_pred_train-y_train)*100<=5)
print("t_score train :", train_score)

ids = np.arange(1, len(predictions) + 1)

# Create a DataFrame for the output
output_df = pd.DataFrame({

    'ID': ids,

    'PURITY': predictions[:,0]

})

# Save the DataFrame to a CSV file
output_df.to_csv('predictions.csv', index=False)

n_comp 77
n_comp 77
y_pred contains NaN: False
t_score train : 1.0


In [ ]:
y_pred_train

In [ ]:
y_train